In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # 2. Camada Bronze - Ingestão dos Dados de Filmes
# MAGIC 
# MAGIC **Objetivo:** Ler o arquivo CSV bruto da Landing Zone e salvá-lo como uma tabela Delta na camada Bronze.

# COMMAND ----------

from pyspark.sql.functions import current_timestamp, input_file_name

# Define o catálogo e o schema a serem usados
spark.sql("USE CATALOG movies_catalog")
spark.sql("USE SCHEMA bronze")

# COMMAND ----------

# Caminho para o arquivo no volume
landing_path = "/Volumes/movies_catalog/bronze/landing_zone_movies/top_100_movies_full_best_effort.csv"
bronze_table_name = "movies_raw"

# Leitura do arquivo CSV bruto
df_raw = spark.read.format("csv") \
                   .option("header", "true") \
                   .option("inferSchema", "true") \
                   .load(landing_path)

# Adiciona metadados de controle
df_bronze = df_raw.withColumn("data_ingestao", current_timestamp()) \
                  .withColumn("arquivo_origem", input_file_name())

# Salva como uma tabela Delta na camada Bronze
df_bronze.write.mode("overwrite") \
         .option("overwriteSchema", "true") \
         .saveAsTable(bronze_table_name)

print(f"Tabela '{bronze_table_name}' criada com sucesso na camada Bronze.")
display(spark.table(bronze_table_name))